In [1]:
import numpy as np
import tensorflow as tf
import pickle as pkl
np.random.seed(0)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
np.random.seed(1)

In [4]:
datapath="C:/Users/aiden/Downloads/ProjectData/FeatureData/"
with open(datapath+"Train", "rb") as f:
    X_train, Y_train, vocab = pkl.load(f)
with open(datapath+"Test", "rb") as f:
    X_test, Y_test, _ = pkl.load(f)


In [7]:
Y_test[0]

['y',
 'Right',
 '=',
 'Right',
 'A',
 'Right',
 'x',
 'Right',
 '+',
 'Right',
 'A',
 'Sup',
 '2']

In [9]:
def data_to_index (data):
    return [[vocab[w] for w in arr] for arr in data]

In [12]:
vocab

{'Right': 0,
 'Sub': 1,
 'Sup': 2,
 'Above': 3,
 'Below': 4,
 'Inside': 5,
 'NoRel': 6,
 'y': 7,
 '=': 8,
 'A': 9,
 'x': 10,
 '+': 11,
 '2': 12,
 'B': 13,
 'n': 14,
 '(': 15,
 '1': 16,
 '-': 17,
 ')': 18,
 '0': 19,
 '\\lt': 20,
 'C': 21,
 '\\times': 22,
 '\\gt': 23,
 '\\forall': 24,
 '\\in': 25,
 'COMMA': 26,
 'a': 27,
 'b': 28,
 'c': 29,
 '\\exists': 30,
 'Y': 31,
 'j': 32,
 '\\cos': 33,
 'i': 34,
 '[': 35,
 ']': 36,
 '\\gamma': 37,
 '\\alpha': 38,
 '\\ldots': 39,
 'd': 40,
 't': 41,
 '\\{': 42,
 '\\}': 43,
 'k': 44,
 '\\leq': 45,
 'z': 46,
 '\\pi': 47,
 'f': 48,
 'e': 49,
 '\\lim': 50,
 '\\rightarrow': 51,
 '\\infty': 52,
 'g': 53,
 '5': 54,
 '9': 55,
 '3': 56,
 '\\sum': 57,
 '8': 58,
 '7': 59,
 '4': 60,
 '\\neq': 61,
 '\\tan': 62,
 '\\sqrt': 63,
 '\\int': 64,
 '\\beta': 65,
 '\\pm': 66,
 '6': 67,
 '\\theta': 68,
 '/': 69,
 'X': 70,
 '\\div': 71,
 '.': 72,
 'p': 73,
 '\\sin': 74,
 '\\phi': 75,
 'r': 76,
 '\\geq': 77,
 '\\log': 78,
 'm': 79,
 '!': 80,
 'F': 81,
 'P': 82,
 'v': 83,
 'w

In [13]:
Y_train[0]

['B',
 'Sub',
 'n',
 'NoRel',
 '(',
 'Right',
 '1',
 'Right',
 '-',
 'Right',
 'x',
 'Right',
 ')',
 'Right',
 '=',
 'Right',
 '(',
 'Right',
 '-',
 'Right',
 '1',
 'Right',
 ')',
 'Sup',
 'n',
 'NoRel',
 'B',
 'Sub',
 'n',
 'NoRel',
 '(',
 'Right',
 'x',
 'Right',
 ')']

In [11]:
data_to_index(Y_train)[0]

[13,
 1,
 14,
 6,
 15,
 0,
 16,
 0,
 17,
 0,
 10,
 0,
 18,
 0,
 8,
 0,
 15,
 0,
 17,
 0,
 16,
 0,
 18,
 2,
 14,
 6,
 13,
 1,
 14,
 6,
 15,
 0,
 10,
 0,
 18]

In [ ]:
def MathModel(input_shape, output_dim):
    input = Input(shape=input_shape)
    X = Bidirectional(LSTM(128, return_sequences = True))(input)
    X = Dropout(0.5)(X)
    X = Bidirectional(LSTM(128, return_sequences = True))(X)
    X = Dropout(0.5)(X)
    X = Bidirectional(LSTM(128, return_sequences = True))(X)
    X = Dropout(0.5)(X)
    X = Dense(output_dim+1)(X)
    X = Activation('softmax')(X)
    
    model = Model(inputs=input,outputs=X)
    
    return model

In [ ]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [ ]:
model.compile(optimizer='adam', loss = CTCLoss, metrics = ['accuracy'])